# Testing and Code Quality

This notebook covers running tests, linting, formatting, and type checking for the Azure Document Intelligence pipeline.

## Setup

Navigate to project root and ensure dependencies are installed.

In [ ]:
# Navigate to project root

Set-Location -Path "E:\Repos\GitHub\MyDemoRepos\FormExtraction"

# Verify UV is available
uv --version

Write-Host "`nProject root: $(Get-Location)" -ForegroundColor Cyan

## Unit Tests

Run unit tests that don't require deployed Azure resources.

In [ ]:
# Run all unit tests
Write-Host "Running unit tests..." -ForegroundColor Cyan
uv run pytest tests/unit/ -v

In [ ]:
# Run unit tests with coverage report
Write-Host "Running unit tests with coverage..." -ForegroundColor Cyan
uv run pytest tests/unit/ --cov=src/functions --cov-report=html --cov-report=term

Write-Host "`nHTML coverage report saved to: htmlcov/index.html" -ForegroundColor Green

In [ ]:
# Open coverage report in browser (Windows)
if (Test-Path "htmlcov/index.html") {
    Start-Process "htmlcov/index.html"
    Write-Host "Opened coverage report in browser" -ForegroundColor Green
} else {
    Write-Host "Coverage report not found. Run tests with --cov first." -ForegroundColor Yellow
}

In [ ]:
# Run specific test file
Write-Host "Running specific test file..." -ForegroundColor Cyan

# List available test files
Write-Host "Available test files:" -ForegroundColor Yellow
Get-ChildItem -Path tests/unit/ -Filter "test_*.py" | ForEach-Object { Write-Host "  - $($_.Name)" }

# Run a specific test file (uncomment and modify as needed)
# uv run pytest tests/unit/test_config.py -v

## Integration Tests

Integration tests require deployed Azure resources and a configured `.env` file.

In [ ]:
# Check if .env file exists
if (Test-Path ".env") {
    Write-Host ".env file found" -ForegroundColor Green
    
    # Run integration tests
    Write-Host "`nRunning integration tests..." -ForegroundColor Cyan
    Write-Host "Note: This requires deployed Azure resources" -ForegroundColor Yellow
    
    $env:RUN_INTEGRATION_TESTS = "1"
    uv run pytest tests/integration/ -v
} else {
    Write-Host ".env file not found" -ForegroundColor Red
    Write-Host "Create .env file with Azure resource details before running integration tests" -ForegroundColor Yellow
    Write-Host "See 03-Local-Development.ipynb for setup instructions" -ForegroundColor Yellow
}

## Linting with Ruff

Check code style and quality with Ruff.

In [ ]:
# Run Ruff linter
Write-Host "Running Ruff linter..." -ForegroundColor Cyan
uv run ruff check src/ tests/

In [ ]:
# Auto-fix lint issues
Write-Host "Auto-fixing lint issues..." -ForegroundColor Cyan
uv run ruff check --fix src/ tests/

Write-Host "`nLint issues fixed where possible" -ForegroundColor Green

## Code Formatting with Ruff

In [ ]:
# Check formatting (dry run)
Write-Host "Checking code formatting..." -ForegroundColor Cyan
uv run ruff format --check src/ tests/

In [ ]:
# Apply formatting
Write-Host "Formatting code..." -ForegroundColor Cyan
uv run ruff format src/ tests/

Write-Host "`nCode formatted" -ForegroundColor Green

## Type Checking with MyPy

In [ ]:
# Run MyPy type checker
Write-Host "Running MyPy type checker..." -ForegroundColor Cyan
uv run mypy src/functions/ --ignore-missing-imports

## Bicep Validation

Validate Bicep infrastructure templates.

In [ ]:
# Validate Bicep syntax
Write-Host "Validating Bicep templates..." -ForegroundColor Cyan

# Build main.bicep (validates syntax)
az bicep build --file infra/main.bicep --stdout | Out-Null

if ($LASTEXITCODE -eq 0) {
    Write-Host "main.bicep - Valid" -ForegroundColor Green
} else {
    Write-Host "main.bicep - Invalid" -ForegroundColor Red
}

# Validate each module
Get-ChildItem -Path infra/modules/*.bicep | ForEach-Object {
    az bicep build --file $_.FullName --stdout 2>&1 | Out-Null
    if ($LASTEXITCODE -eq 0) {
        Write-Host "$($_.Name) - Valid" -ForegroundColor Green
    } else {
        Write-Host "$($_.Name) - Invalid" -ForegroundColor Red
    }
}

In [ ]:
# Validate parameter files
Write-Host "Validating parameter files..." -ForegroundColor Cyan

Get-ChildItem -Path infra/parameters/*.bicepparam | ForEach-Object {
    az bicep build-params --file $_.FullName --stdout 2>&1 | Out-Null
    if ($LASTEXITCODE -eq 0) {
        Write-Host "$($_.Name) - Valid" -ForegroundColor Green
    } else {
        Write-Host "$($_.Name) - Invalid" -ForegroundColor Red
    }
}

## Run All Checks

Run all quality checks in sequence.

In [ ]:
Write-Host "========================================" -ForegroundColor Cyan
Write-Host "   Running All Quality Checks" -ForegroundColor Cyan
Write-Host "========================================" -ForegroundColor Cyan

$results = @()

# Linting
Write-Host "`n[1/4] Linting..." -ForegroundColor Yellow
uv run ruff check src/ tests/ 2>&1 | Out-Null
$results += @{ Check = "Ruff Lint"; Status = if ($LASTEXITCODE -eq 0) { "PASS" } else { "FAIL" } }

# Formatting
Write-Host "[2/4] Formatting..." -ForegroundColor Yellow
uv run ruff format --check src/ tests/ 2>&1 | Out-Null
$results += @{ Check = "Ruff Format"; Status = if ($LASTEXITCODE -eq 0) { "PASS" } else { "FAIL" } }

# Type checking
Write-Host "[3/4] Type checking..." -ForegroundColor Yellow
uv run mypy src/functions/ --ignore-missing-imports 2>&1 | Out-Null
$results += @{ Check = "MyPy Types"; Status = if ($LASTEXITCODE -eq 0) { "PASS" } else { "FAIL" } }

# Unit tests
Write-Host "[4/4] Unit tests..." -ForegroundColor Yellow
uv run pytest tests/unit/ -q 2>&1 | Out-Null
$results += @{ Check = "Unit Tests"; Status = if ($LASTEXITCODE -eq 0) { "PASS" } else { "FAIL" } }

# Summary
Write-Host "`n========================================" -ForegroundColor Cyan
Write-Host "   Results" -ForegroundColor Cyan
Write-Host "========================================" -ForegroundColor Cyan

$results | ForEach-Object {
    $color = if ($_.Status -eq "PASS") { "Green" } else { "Red" }
    Write-Host "  $($_.Check): $($_.Status)" -ForegroundColor $color
}

$passed = ($results | Where-Object { $_.Status -eq "PASS" }).Count
$total = $results.Count

Write-Host "`n  Total: $passed/$total passed" -ForegroundColor $(if ($passed -eq $total) { "Green" } else { "Yellow" })

## Test Configuration

The project uses pytest with the following configuration (from `pyproject.toml`):

```toml
[tool.pytest.ini_options]
testpaths = ["tests"]
python_files = "test_*.py"
python_functions = "test_*"
addopts = "-v --tb=short"
```

### Test Markers

- `@pytest.mark.integration` - Integration tests requiring Azure resources
- `@pytest.mark.slow` - Tests that take longer to run

### Environment Variables for Testing

- `RUN_INTEGRATION_TESTS=1` - Enable integration tests